# How color is related to sentiment and object in letters

In [1]:
import pandas as pd # library for data analysis
import re
import shutil
import webcolors
import spacy
from spacy import displacy
from collections import Counter
from spacy.matcher import Matcher
from spacy.util import filter_spans 

nlp = spacy.load('en_core_web_sm')

In [2]:
letters = pd.read_csv("letter_raw.csv")
letters.head()

,id,header,content,sketch,from,to,place,date
0,1,"To Theo van Gogh. The Hague, Sunday, 29 Septem...","The Hague, 29 September 1872. My dear Theo, Th...",False,Vincent van Gogh,Theo van Gogh,The Hague,29 September 1872.
1,2,"To Theo van Gogh. The Hague, Friday, 13 Decemb...","The Hague, 13 December 1872. My dear Theo, Tha...",False,Vincent van Gogh,Theo van Gogh,The Hague,13 December 1872.
2,3,"To Theo van Gogh. The Hague, mid-January 1873.","The Hague, January 1873 My dear Theo, I heard ...",False,Vincent van Gogh,Theo van Gogh,The Hague,mid-January 1873.
3,4,"To Theo van Gogh. The Hague, Tuesday, 28 Janua...","The Hague, 28 Jan. 1873 My dear Theo, It’s goo...",False,Vincent van Gogh,Theo van Gogh,The Hague,28 January 1873.
4,5,"To Theo van Gogh. The Hague, Monday, 17 March ...","The Hague, 17 March 1873 My dear Theo, It’s ti...",False,Vincent van Gogh,Theo van Gogh,The Hague,17 March 1873.


### Construct color list

In [3]:
webcolors.css3_names_to_hex.keys()

dict_keys(['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lights

In [4]:
# construct the list manually from the webcolors and search function on the letter website
# not include words like dark/light/pale, as they are description of color
color_words = ['colour', 'tint', 'purplish', 'yellowish', 'azure', 'black',
              'blue', 'brown', 'crimson', 'gold', 'silver', 'ivory', 'indigo', 'green',
              'grey', 'orange', 'pink', 'purple', 'red', 'violet', 'white', 'yellow'] 

### Get sentences contain keywords

In [5]:
'''
for each letter, loop over the content sentence by sentence, 
get sentences contain key words
'''
sents_w_keyword = []

for i, row in letters.iterrows():
    doc = nlp(row.content)
    for sentence in doc.sents:
        sentence = sentence.text
        
        for word in color_words:
            if word in sentence.lower().split():
                #print(word, sentence)
                sents_w_keyword.append((row.id, sentence))
                break # find one keyword in sentence is enough

In [6]:
sents_w_keyword[1]

(10,
 'The bell, in the grey tower entwined with black-green ivy, Was now silent.')

In [7]:
len(sents_w_keyword)

1738

### NLP

In [9]:
def takeStart(ne):
    return ne[0]
    
# noun phrase: (DET)?(ADJ)*(NOUN)*Noun
#[{"POS":"DET","OP":"?"}, {"POS":"ADJ","OP":"*"}, {"POS":"NOUN","OP":"+"}]  
matcher = Matcher(nlp.vocab)
matcher.add("NP", [[{"POS":"DET","OP":"?"}, {"POS":"ADJ","OP":"*"}, {"POS":"NOUN","OP":"+"}]])


results = [] #[(letter id, colorword, NP)...] FINAL RESULT


# for each sentence
for comp in sents_w_keyword:
    #### first, match the pattern of noun phrase
    letter_id = comp[0]
    doc = nlp(comp[1])
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches]
    filtered = filter_spans(spans)   

    savedNP = [] #[(start, end, NP)]
    for _, start, end in matches:
        if(doc[start:end] in filtered):
            savedNP.append((start, end, doc[start:end]))

    savedNP.sort(key=takeStart)  
    
    
    i = 0
    for ent in doc:
        if ent.text in color_words:
            np_before_dist = 0 # distance to the end of the NP before color word
            np_after_dist = 0 # distance to the start of the NP after color word

            closestNP = None
            beforeNP = None
            afterNP = None
            for np in savedNP:
                # dont consider if the other color words are recognized as noun
                if(np[2].text in color_words):
                    continue
                # need to consider the situation when the color word is part of the NP! Then it must be an adjective!
                if(np[0]<=i and np[1]>i and ent.pos_=="ADJ"):
                    closestNP = np[2] # find the NP! break!
                    break

                before_dist = i-np[1] # distance of color word and the NP before it
                after_dist = i-np[0] # distance of color word and the NP after it
                if(before_dist>=0):
                    np_before_dist = before_dist
                    beforeNP = np[2]
                if(after_dist<0):
                    np_after_dist = after_dist 
                    afterNP = np[2]
                    break # find closest NP after the color word, for loop can be ended

            if(closestNP == None):
                # compare the distance of NP from before and after the color word
                if(np_before_dist>np_after_dist):
                    closestNP = afterNP
                else:
                    closestNP = beforeNP

            results.append((letter_id, ent, closestNP)) 
        i += 1
    

In [10]:
results

[(10, gold, the evening sun),
 (10, grey, tower),
 (10, black, green ivy),
 (10, green, green ivy),
 (10, brown, the windmill),
 (10, purple, the west),
 (10, yellow, the west),
 (10, grey, the west),
 (11, gold, the evening sun),
 (11, grey, tower),
 (11, black, green ivy),
 (11, green, green ivy),
 (11, brown, the windmill),
 (11, purple, the west),
 (11, yellow, the west),
 (11, grey, the west),
 (12, green, vallies),
 (12, green, rivers),
 (12, silver, breastplate),
 (13, blue, blue wallpaper),
 (13, green, a green border),
 (13, black, black pigs),
 (14, black, silk),
 (14, grey, silk),
 (14, grey, sea),
 (30, yellow, a dull yellow sandy road),
 (30, green, moss),
 (30, brown, a grey sky),
 (30, grey, a grey sky),
 (30, white, the horizon),
 (31, red, brownish red houses),
 (31, grey, grey roofs),
 (31, white, yellow doors),
 (31, yellow, yellow doors),
 (31, white, a large white drawbridge),
 (31, white, white horses),
 (31, black, white caps),
 (31, white, white caps),
 (31, bla

In [10]:
len(results)

3575